In [1]:
import pandas as pd
import numpy as np
import requests
from datetime import datetime, timezone, timedelta
from io import StringIO
import geohash2
import matplotlib.pyplot as plt
import seaborn as sns
import sys
import math
# Gloabl seaborn Theme
sns.set_theme(style="whitegrid", palette="pastel")

# Extraction - PM25 - NowCast AQI

---

## Air Quality Index - AQI - Basics

### What is the U.S. Air Quality Index - AQI?

The U.S. AQI is EPA’s index for reporting air quality.

[AQI - Basics](https://www.airnow.gov/aqi/aqi-basics/)

[Technical Assistance Document for the Reporting of Daily Air Quality – the Air Quality Index - AQI](https://www.airnow.gov/sites/default/files/2020-05/aqi-technical-assistance-document-sept2018.pdf)

| Using Table 5, find the two breakpoints that contain the concentration. | AQI for Ozone and Particle Pollution |
| --- | --- |
| <img src="breakpoints_aqi.png" alt="Breakpoints for the AQI" width="auto"/> | <img src="categories_aqi.png" alt="Categories for the AQI" width="auto"/> |


### Calculating the AQI

**How do I Calculate the AQI from pollutant concentration data?**

The AQI is the highest value calculated for each pollutan as follows:

    A. Identify the highest concentration among all of the monitors within each reporting area and truncate as follow:

<img src="pollutants.png" alt="Pollutants" width="auto"/>

    B. Using Table 5, find the two breakpoints tha contain the concentration.

    C. Using Equation 1, calculate the index

<img src="equation1.png" alt="Equation 1" width="auto"/>

    D. Round the index to the nearest integer


### Constants

In [2]:
# Levels of Concern
GOOD = 0
MODERATE = 1
UNHEALTHY_FOR_SENSITIVE_GROUPS = 2
UNHEALTHY = 3
VERY_UNHEALTHY = 4
HAZARDOUS = 5
# AQI Levels
AQI_LEVELS = (
    (GOOD, 'Good', [0, 50]), # 0 - 50
    (MODERATE, 'Moderate', [51, 100]), # 51 - 100
    (UNHEALTHY_FOR_SENSITIVE_GROUPS, 'Unhealthy for sensitive groups', [101, 150]), # 101 - 150
    (UNHEALTHY, 'Unhealthy', [151, 200]), # 151 - 200
    (VERY_UNHEALTHY, 'Very Unhealthy', [201, 300]), # 201 - 300
    (HAZARDOUS, 'Hazardous', [301, sys.maxsize]), # 301 - higher
)
# PM25 Breakpoints Values
PM25_BREAKPOINTS = (
    (GOOD, 'Good', [0.0, 12.0]), # 0.0 - 12.0, Good
    (MODERATE, 'Moderate', [12.1, 35.4]), # 12.1 - 35.4, Moderate
    (UNHEALTHY_FOR_SENSITIVE_GROUPS, 'Unhealthy for sensitive groups', [35.5, 55.4]), # 35.5 - 55.4, Unhealthy for sensitive groups
    (UNHEALTHY, 'Unhealthy', [55.5, 150.4]), # 55.5 - 150.4, Unhealthy
    (VERY_UNHEALTHY, 'Very Unhealthy', [150.5, 250.4]), # 150.5 - 250.4, Very Unhealthy
    (HAZARDOUS, 'Hazardous', [250.5, 350.4]), # 250.5 - 350.4, Hazardous
    (HAZARDOUS, 'Hazardous', [350.5, 500.4]), # 350.5 - 500.4, Hazardous
)
# Max PM25 Value
MAX_PM25_VALUE = 500.5

### Equation 1

In [3]:
# Define Equation 1 to Calculate AQI Value
#
# AQI = AQI for pollutant p
# Cp = The truncated concentrattion of pollutant p
# BP_hi = The concentration breakpoint that is greater than or equal to Cp
# BP_lo = The concentration breakpoint that is less than or equal to Cp
# AQI_hi = The AQI value corresponding to BP_hi
# AQI_lo = The AQI value corresponding to BP_lo
#
def equation1(Cp):
    # AQI Value to Concentration of Pollutant P
    AQI = None
    # Variables
    BP_hi = None
    BP_lo = None
    AQI_hi = None
    AQI_lo = None

    # Get BP_hi and BP_lo
    for bp in PM25_BREAKPOINTS:
        #print('bp:', bp)
        if (Cp >= bp[2][0]) and (Cp <= bp[2][-1]):
            BP_hi = bp[2][-1]
            BP_lo = bp[2][0]
            AQI_hi = AQI_LEVELS[bp[0]][2][-1]
            AQI_lo = AQI_LEVELS[bp[0]][2][0]
            break

    AQI = math.ceil((((AQI_hi - AQI_lo) / (BP_hi - BP_lo)) * (Cp - BP_lo)) + AQI_lo)
    #print('Cp:', Cp, 'BP_hi:', BP_hi, 'BP_lo:', BP_lo, 'AQI_hi:', AQI_hi, 'AQI_lo:', AQI_lo, 'AQI:', AQI)
    #print('Cp:', Cp, 'AQI:', AQI)

    return AQI

In [5]:
# Equation 1 to calculate the AQI value
pm25_value = 35.9
aqi_value = equation1(35.9)
print('PM25:', pm25_value, 'AQI:', aqi_value)

PM25: 35.9 AQI: 102


---